## Data

In [ ]:
!pip install catboost
!pip install fastai
!pip install gensim
!pip install huggingface-hub
!pip install hyperopt
!pip install keras
!pip install Keras-Preprocessing
!pip install konlpy
!pip install lightgbm
!pip install MarkupSafe
!pip install mecab-python
!pip install mlxtend
!pip install opencv-python
!pip install optuna
!pip install plotly
!pip install PyMySQL
!pip install scikit-learn
!pip install spacy
!pip install SQLAlchemy
!pip install tensorboard
!pip install tensorflow
!pip install tensorflow-datasets
!pip install torch
!pip install torchaudio
!pip install torchvision
!pip install wandb
!pip install xgboost
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.1 MB/s 
     |█████████████████████████████

In [ ]:
#pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/BIZ&AI_PAPER_11기/세희/memento/KoBERT-KorQuAD-master

/content/drive/.shortcut-targets-by-id/1cPIIU0uxZfRWO_8VJ5lEbuVRPz7AvhgA/BIZ&AI_PAPER_11기/세희/memento/KoBERT-KorQuAD-master


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/1cPIIU0uxZfRWO_8VJ5lEbuVRPz7AvhgA/BIZ&AI_PAPER_11기/세희/memento/KoBERT-KorQuAD-master


In [ ]:
!ls

data		  LICENSE      README.md	 run_squad.py
evaluate_v1_0.py  __pycache__  requirements.txt  tokenization_kobert.py


In [ ]:
import pandas as pd
import re, json, glob, argparse
from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/BIZ&AI_PAPER_11기/세희/memento/dataset.xlsx')
df

,Unnamed: 0,time,location,text
0,0,2022-06-26 01:43:56,정통집,평일에 열심히 일하고 바쁘게 살았다면 주말에는 그저 누워서 하릴없이 남편이랑 빈둥빈둥
1,1,2021-11-18 05:25:00,설빙,"집 청소 하고 비워진 냉장고 채우고 재활용품, 쓰레기 정리하고 이불 빨래 하고 이..."
2,2,2022-08-21 11:32:47,정통집,그럼에도 주말 이틀을 기다리면서 평일을 또 열심히 살아가곤 하지
3,3,2022-08-28 08:48:37,와플대학,"퇴근하고나면 모니터를 보기도 싫을 정도로 눈이 빽빽하고 힘든데, 남편이랑 누워서 ..."
4,4,2020-05-07 21:49:13,마쿠 마라탕,\n주말 이틀간 에너지를 가득 채우는 한 주였다
...,...,...,...,...
16285,16285,2020-12-15 20:06:58,설빙,\n가족은 만나서 즐거워야 또 만난다
16286,16286,2021-01-07 20:42:54,롤링파스타,\n또 만나고 싶은거다
16287,16287,2021-11-02 11:13:23,끄트머리집,"\n남은 연휴,\n서산에서 보내온\n사과와 배를 까먹으며\n다음 만남의 계획을 세워본다"
16288,16288,2022-09-26 17:19:30,은화수식당,\n내 집에 왔다~\n내 집이 좋다~\n명절은 좋지만\n끝나면 더좋아


## Preprocessing

In [ ]:
WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [ ]:
def tokenize(content, token_min_len=2, token_max_len=100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)  # remove email pattern
    content = re.sub(URL_PATTERN, ' ', content) # remove url pattern
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)  # remove unnecessary chars
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTIPLE_SPACES, ' ', content)
    tokens = content.replace(", )", "").split(" ")
    result = []
    for token in tokens:
        if not token.startswith('_'):
            token_candidate = to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token))
        else:
            token_candidate = ""
        if len(token_candidate) > 0:
            result.append(token_candidate)
    return result

In [ ]:
tokenize(str(df['text'][500]))

['이날도', '오전부터', '좀', '바빠서', '오후운동', '나갔어요']

In [ ]:
df['text'][500]

'\n이날도 오전부터 좀 바빠서 오후운동 나갔어요'

In [ ]:
tokenize(str(df['text'][16000]))

['다꾸하다가',
 '마테통',
 '엎어서',
 '화가',
 '엄청났다',
 'ㅋㅋㅋㅋㅋ',
 '하지만',
 '바로',
 '치우지도',
 '않고',
 '사진부터찍는',
 'sns중독자',
 '오랜만에',
 '동숲했따',
 '다시',
 '섬',
 '열심히',
 '꾸며야하는데',
 '리셋이후에',
 '큼직하게는',
 '꾸몄는데',
 '아직',
 '세세하게는',
 '못꾸몄다',
 '레시피가',
 '없어',
 '리셋한거',
 '너무너무',
 '후회중']

In [ ]:
df['text'][16000]

'다꾸하다가 마테통 엎어서\n화가 엄청났다 ㅋㅋㅋㅋㅋ\n하지만 바로 치우지도 않고 사진부터찍는\nsns중독자\n오랜만에 동숲했따\n다시 섬 열심히 꾸며야하는데\n리셋이후에 큼직하게는 꾸몄는데\n아직 세세하게는 못꾸몄다\n레시피가 없어\n리셋한거 너무너무 후회중'

In [ ]:
df["pre_text"] = ""

In [ ]:
df['pre_text'] = df['text'].apply(lambda text: tokenize(text))
df['pre_text']

0        [평일에, 열심히, 일하고, 바쁘게, 살았다면, 주말에는, 그저, 누워서, 하릴없이...
1        [집, 청소, 하고, 비워진, 냉장고, 채우고, 재활용품,, 쓰레기, 정리하고, 이...
2            [그럼에도, 주말, 이틀을, 기다리면서, 평일을, 또, 열심히, 살아가곤, 하지]
3        [퇴근하고나면, 모니터를, 보기도, 싫을, 정도로, 눈이, 빽빽하고, 힘든데,, 남...
4                         [주말, 이틀간, 에너지를, 가득, 채우는, 한, 주였다]
                               ...                        
16285                             [가족은, 만나서, 즐거워야, 또, 만난다]
16286                                       [또, 만나고, 싶은거다]
16287    [남은, 연휴,, 서산에서, 보내온, 사과와, 배를, 까먹으며, 다음, 만남의, 계...
16288         [내, 집에, 왔다~, 내, 집이, 좋다~, 명절은, 좋지만, 끝나면, 더좋아]
16289                                    [#명절, #블챌, #주간일기]
Name: pre_text, Length: 16290, dtype: object

In [ ]:
df

,Unnamed: 0,time,location,text,pre_text
0,0,2022-06-26 01:43:56,정통집,평일에 열심히 일하고 바쁘게 살았다면 주말에는 그저 누워서 하릴없이 남편이랑 빈둥빈둥,"[평일에, 열심히, 일하고, 바쁘게, 살았다면, 주말에는, 그저, 누워서, 하릴없이..."
1,1,2021-11-18 05:25:00,설빙,"집 청소 하고 비워진 냉장고 채우고 재활용품, 쓰레기 정리하고 이불 빨래 하고 이...","[집, 청소, 하고, 비워진, 냉장고, 채우고, 재활용품,, 쓰레기, 정리하고, 이..."
2,2,2022-08-21 11:32:47,정통집,그럼에도 주말 이틀을 기다리면서 평일을 또 열심히 살아가곤 하지,"[그럼에도, 주말, 이틀을, 기다리면서, 평일을, 또, 열심히, 살아가곤, 하지]"
3,3,2022-08-28 08:48:37,와플대학,"퇴근하고나면 모니터를 보기도 싫을 정도로 눈이 빽빽하고 힘든데, 남편이랑 누워서 ...","[퇴근하고나면, 모니터를, 보기도, 싫을, 정도로, 눈이, 빽빽하고, 힘든데,, 남..."
4,4,2020-05-07 21:49:13,마쿠 마라탕,\n주말 이틀간 에너지를 가득 채우는 한 주였다,"[주말, 이틀간, 에너지를, 가득, 채우는, 한, 주였다]"
...,...,...,...,...,...
16285,16285,2020-12-15 20:06:58,설빙,\n가족은 만나서 즐거워야 또 만난다,"[가족은, 만나서, 즐거워야, 또, 만난다]"
16286,16286,2021-01-07 20:42:54,롤링파스타,\n또 만나고 싶은거다,"[또, 만나고, 싶은거다]"
16287,16287,2021-11-02 11:13:23,끄트머리집,"\n남은 연휴,\n서산에서 보내온\n사과와 배를 까먹으며\n다음 만남의 계획을 세워본다","[남은, 연휴,, 서산에서, 보내온, 사과와, 배를, 까먹으며, 다음, 만남의, 계..."
16288,16288,2022-09-26 17:19:30,은화수식당,\n내 집에 왔다~\n내 집이 좋다~\n명절은 좋지만\n끝나면 더좋아,"[내, 집에, 왔다~, 내, 집이, 좋다~, 명절은, 좋지만, 끝나면, 더좋아]"


## KorQuAD
> LG CNS가 구축한 한국어 기계 독해 dataset

In [ ]:
df['pre_text']

0        [평일에, 열심히, 일하고, 바쁘게, 살았다면, 주말에는, 그저, 누워서, 하릴없이...
1        [집, 청소, 하고, 비워진, 냉장고, 채우고, 재활용품,, 쓰레기, 정리하고, 이...
2            [그럼에도, 주말, 이틀을, 기다리면서, 평일을, 또, 열심히, 살아가곤, 하지]
3        [퇴근하고나면, 모니터를, 보기도, 싫을, 정도로, 눈이, 빽빽하고, 힘든데,, 남...
4                         [주말, 이틀간, 에너지를, 가득, 채우는, 한, 주였다]
                               ...                        
16285                             [가족은, 만나서, 즐거워야, 또, 만난다]
16286                                       [또, 만나고, 싶은거다]
16287    [남은, 연휴,, 서산에서, 보내온, 사과와, 배를, 까먹으며, 다음, 만남의, 계...
16288         [내, 집에, 왔다~, 내, 집이, 좋다~, 명절은, 좋지만, 끝나면, 더좋아]
16289                                    [#명절, #블챌, #주간일기]
Name: pre_text, Length: 16290, dtype: object

In [ ]:
from transformers import BertModel
from tokenization_kobert import KoBertTokenizer

model = BertModel.from_pretrained('monologg/kobert')
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import os

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [ ]:
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # F1 by character
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

In [ ]:
def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

In [ ]:
def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [ ]:
def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    return {'exact_match': exact_match, 'f1': f1}

In [ ]:
def eval_during_train(args):
    expected_version = 'KorQuAD_v1.0'

    dataset_file = os.path.join(args.data_dir, args.predict_file)
    prediction_file = os.path.join(args.output_dir, 'predictions_.json')

    with open(dataset_file) as dataset_f:
        dataset_json = json.load(dataset_f)
        read_version = "_".join(dataset_json['version'].split("_")[:-1])
        if (read_version != expected_version):
            print('Evaluation expects ' + expected_version +
                  ', but got dataset with ' + read_version,
                  file=sys.stderr)
        dataset = dataset_json['data']
    with open(prediction_file) as prediction_f:
        predictions = json.load(prediction_f)

    return evaluate(dataset, predictions)

In [ ]:
#전처리 적용
normalize_answer(str(df['text'][900]))

'31 월 30년 노후생활 30년 준비하자 사회생활을 준비하기 위해 초중고 대학을 나와 열심히 직장을 위해 준비하면서 30년 40년 살게 될 노후에 대해선 천천히 하지 뭐 하는 생각이 많다'

In [ ]:
df['text'][900]

' 31 (월)\n30년 노후생활 30년 준비하자\n사회생활을 준비하기 위해\n초중고 대학을 나와 열심히\n직장을 위해 준비하면서\n30년, 40년 살게 될 노후에 대해선\n천천히 하지 뭐 하는 생각이 많다'

In [ ]:
f1_score(str(df['text'][900]), str(10))

0.05063291139240507

In [ ]:
exact_match_score(str(df['text'][900]), str(10))

False

In [ ]:
#metric_max_over_ground_truths(metric_fn, str(df['text'][900]), str(10))

In [ ]:
df['pre_text'] = df['text'].apply(lambda text: tokenize(text))
df['pre_text']

0        [평일에, 열심히, 일하고, 바쁘게, 살았다면, 주말에는, 그저, 누워서, 하릴없이...
1        [집, 청소, 하고, 비워진, 냉장고, 채우고, 재활용품,, 쓰레기, 정리하고, 이...
2            [그럼에도, 주말, 이틀을, 기다리면서, 평일을, 또, 열심히, 살아가곤, 하지]
3        [퇴근하고나면, 모니터를, 보기도, 싫을, 정도로, 눈이, 빽빽하고, 힘든데,, 남...
4                         [주말, 이틀간, 에너지를, 가득, 채우는, 한, 주였다]
                               ...                        
16285                             [가족은, 만나서, 즐거워야, 또, 만난다]
16286                                       [또, 만나고, 싶은거다]
16287    [남은, 연휴,, 서산에서, 보내온, 사과와, 배를, 까먹으며, 다음, 만남의, 계...
16288         [내, 집에, 왔다~, 내, 집이, 좋다~, 명절은, 좋지만, 끝나면, 더좋아]
16289                                    [#명절, #블챌, #주간일기]
Name: pre_text, Length: 16290, dtype: object